In [1]:
#!/usr/bin/env python3
import sys
import copy
sys.path.append('../../')

from __future__ import print_function
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, TeV
from smodels.matching.theoryPrediction import theoryPredictionsFor,TheoryPredictionsCombiner
from smodels.matching import theoryPrediction
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.base.smodelsLogging import setLogLevel,logger
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.tools.printers.masterPrinter import MPrinter
from smodels.tools import ioObjects
import time
import os
from collections import OrderedDict
from configparser import ConfigParser, NoSectionError, NoOptionError
setLogLevel("error")

In [2]:
# Set the path to the database
database = Database('unittest')
database.selectExpResults(useNonValidated=False)
databaseVersion = database.databaseVersion

In [3]:
parameterFile='../testPrinters_parameters.ini'
parser = ConfigParser(inline_comment_prefixes=(';',))
ret = parser.read(parameterFile)
parser.set("printer","outputType","log")
parser.set("printer","outputFormat","current")

inputFile = '../testFiles/slha/lightEWinos.slha'
outputDir = '../'

In [4]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
minmassgap = parser.getfloat("parameters", "minmassgap") * GeV

masterPrinter = MPrinter()
masterPrinter.setPrinterOptions(parser)
masterPrinter.setOutPutFiles(os.path.join(
    outputDir, os.path.basename(inputFile)))

In [5]:
masterPrinter.addObj(database)

inputStatus = ioObjects.FileStatus()
if parser.getboolean("options", "checkInput"):
    inputStatus.checkFile(inputFile)

printParameters = []
if parser.has_section('parameters'):
    printParameters += list(parser.items('parameters'))
if parser.has_section('particles'):
    printParameters += list(parser.items('particles'))
if parser.has_section('options'):
    printParameters += list(parser.items('options'))

printParameters = OrderedDict(printParameters)
outputStatus = ioObjects.OutputStatus(inputStatus.status, inputFile,
                                      printParameters, databaseVersion)
masterPrinter.addObj(outputStatus)

In [6]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
promptWidth = None
stableWidth = None
if parser.has_option("particles", "promptWidth"):
    promptWidth = parser.getfloat("particles", "promptWidth")*GeV
if parser.has_option("particles", "stableWidth"):
    stableWidth = parser.getfloat("particles", "stableWidth")*GeV
model.updateParticles(inputFile=inputFile,
                      promptWidth=promptWidth, stableWidth=stableWidth)

In [7]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
smstoplist = decomposer.decompose(model, sigmacut,
                                  massCompress=parser.getboolean(
                                      "options", "doCompress"),
                                  invisibleCompress=parser.getboolean(
                                      "options", "doInvisible"),
                                  minmassgap=minmassgap)


In [8]:
masterPrinter.addObj(smstoplist)

In [9]:
allPredictions = []
combineResults = False
useBest = True
combineResults = parser.getboolean("options", "combineSRs")
allSRs = parser.getboolean("options", "reportAllSRs")
if allSRs:  # If set print out all SRs and skip combination
    useBest = False
    combineResults = False

allPredictions = theoryPredictionsFor(database,smstoplist,
                                             useBestDataset=useBest, combinedResults=combineResults)

if parser.getboolean("options", "computeStatistics"):
    for theoPred in allPredictions:
        theoPred.computeStatistics()

maxcond = parser.getfloat("parameters", "maxcond")
theoryPredictions = theoryPrediction.TheoryPredictionList(
    allPredictions, maxcond)


In [10]:
outputStatus.updateStatus(1)
masterPrinter.addObj(theoryPredictions)

In [11]:
if parser.getboolean("options", "testCoverage"):
    """ Testing coverage of model point, add results to the output file """
    if parser.has_option("options", "coverageSqrts"):
        sqrts = parser.getfloat("options", "coverageSqrts")*TeV
    else:
        sqrts = None
    uncovered = coverage.Uncovered(
        smstoplist, sigmacut=sigmacut, sqrts=sqrts)
    masterPrinter.addObj(uncovered)


In [12]:
if parser.has_option("options", "combineAnas"):
    """ Combine analyses """

    combineAnas = parser.get("options", "combineAnas").replace(" ","").split(",")
    if combineAnas:
        if combineResults is True:
            logger.warning("Combining analyses with signal region combination (combineSRs=True) might significantly reduce CPU performance.")
        combiner = TheoryPredictionsCombiner.selectResultsFrom(theoryPredictions,
                                                               combineAnas)
        # Only compute combination if at least one result was selected
        if combiner is not None:
            combiner.computeStatistics()
            masterPrinter.addObj(combiner)


In [13]:
masterCopy = copy.deepcopy(masterPrinter)
x = masterPrinter.flush()
with open(masterPrinter.Printers['log'].filename,'r') as f:
    lines = f.readlines()
os.remove(masterCopy.Printers['log'].filename)                

In [14]:
print(len(lines))

1198


In [15]:
for l in lines[0:21]:
    if 'Database version' in l:
        continue        
    print(l)

Input status: 1

Decomposition output status: 1 #decomposition was successful

# Input File: ../testFiles/slha/lightEWinos.slha

# checkinput = True

# combineanas = ATLAS-CONF-2013-037,CMS-SUS-13-012

# combinesrs = False

# computestatistics = True

# docompress = True

# doinvisible = True

# experimentalfeatures = False

# maxcond = 0.2

# minmassgap = 5.

# model = share.models.mssm

# ncpus = 1

# promptwidth = 1e-11

# reportallsrs = False

# sigmacut = 10.0

# stablewidth = 1e-25

# testcoverage = True




In [16]:
for l in lines[21:205]:
    print(l)


||                                                        ||

||             Selected Experimental Results              ||

||                                                        ||



Experimental Result ID: ATLAS-SUSY-2016-08

Tx Labels: ['T5Disp']

Sqrts: 1.30E+01

	 -----------------------------

	 SMS tested by analysis:

	    (PV > anyBSM(1),anyBSM(2)), (anyBSM(1) > q,q,MET), (anyBSM(2) > q,q,MET) 


Experimental Result ID: ATLAS-SUSY-2016-081

Tx Labels: ['T5Disp']

Sqrts: 1.30E+01

	 -----------------------------

	 SMS tested by analysis:

	    (PV > gluino(1),gluino(2)), (gluino(1) > q,q,MET), (gluino(2) > q,q,MET) 


Experimental Result ID: ATLAS-SUSY-2018-31

Tx Labels: ['T6bbHH']

Sqrts: 1.30E+01

	 -----------------------------

	 SMS tested by analysis:

	    (PV > anyBSM(1),anyBSM(2)), (anyBSM(1) > anyBSM(3),b), (anyBSM(2) > anyBSM(4),b), (anyBSM(3) > higgs,MET), (anyBSM(4) > higgs,MET) 


Experimental Result ID: CMS-PAS-EXO-16-036

Tx Labels: ['THSCPM1', 'THSCPM2',

In [17]:
for l in lines[205:300]:    
    print(l)


||                                                        ||

||                    Topologies Table                    ||

||                                                        ||



Topology: 110110101000 

Total Global topology weight :

Sqrts: 1.30E+01 [TeV], Weight:3.50E-01 [pb]

Sqrts: 8.00E+00 [TeV], Weight:2.06E-01 [pb]



Total Number of SMS: 4

		 .........................................................................

		 SMS ID: 1

		 SMS: (PV > N1,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,q)

		 Masses: [(N1, 6.81E+01 [GeV]), (C1+/C1-, 1.34E+02 [GeV]), (N1/N1~, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:3.50E-01 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:2.06E-01 [pb]

 		 

		 .........................................................................

		 SMS ID: 2

		 SMS: (PV > N1,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,c)

		 Masses: [(N1, 6.81E+01 [GeV]), (C1+/C1-, 1.34E+02 [GeV]), (N1/N1~, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 

In [18]:
for l in lines[300:400]:    
    print(l)

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:8.28E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.19E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 11

		 SMS: (PV > C1-(1),N2(2)), (C1-(1) > N1~,e-,nu), (N2(2) > N1,c,c)

		 Masses: [(C1-, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.30E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.17E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 12

		 SMS: (PV > C1-(1),N2(2)), (C1-(1) > N1~,e-,nu), (N2(2) > N1,b,b)

		 Masses: [(C1-, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.87E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.45E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 13

		 SM

In [19]:
for l in lines[400:500]:    
    print(l)

		 .........................................................................

		 SMS ID: 23

		 SMS: (PV > C1-(1),C1+(2)), (C1-(1) > N1~,ta-,nu), (C1+(2) > N1,nu,ta+)

		 Masses: [(C1-, 1.34E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.40E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.29E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 24

		 SMS: (PV > C1-(1),C1+(2)), (C1-(1) > N1~,ta-,nu), (C1+(2) > N1,q,q)

		 Masses: [(C1-, 1.34E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:7.70E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.13E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 25

		 SMS: (PV > C1-(1),C1+(2)), (C1-(1) > N1~,ta-,nu), (C1+(2) > N1,q,c)

		 Masses: [(C1-, 1.34E+02 [GeV]), (C1+, 1.34E

In [20]:
for l in lines[500:600]:    
    print(l)

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:7.72E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.14E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 36

		 SMS: (PV > C1+(1),N2(2)), (C1+(1) > N1,nu,mu+), (N2(2) > N1,q,q)

		 Masses: [(C1+, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:1.46E-01 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:8.04E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 37

		 SMS: (PV > C1+(1),N2(2)), (C1+(1) > N1,nu,mu+), (N2(2) > N1,c,c)

		 Masses: [(C1+, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:4.07E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:2.24E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 38

		 SMS:

In [21]:
for l in lines[600:700]:    
    print(l)

		 .........................................................................

		 SMS ID: 48

		 SMS: (PV > C1+/C1-(1),N2(2)), (C1+/C1-(1) > N1/N1~,q,q), (N2(2) > N1,ta-,ta+)

		 Masses: [(C1+/C1-, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1/N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:3.09E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.65E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 49

		 SMS: (PV > C1+/C1-(1),N2(2)), (C1+/C1-(1) > N1/N1~,q,q), (N2(2) > N1,nu,nu)

		 Masses: [(C1+/C1-, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1/N1~, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:1.21E-01 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:6.47E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 50

		 SMS: (PV > C1+/C1-(1),N2(2)), (C1+/C1-(1) > N1/N1~,q,q), (N2(2) > N1,q,q)

		 Masses: [(

In [22]:
for l in lines[700:876]:    
    print(l)

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.54E-01 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.36E-01 [pb]

 		 

		 .........................................................................

		 SMS ID: 61

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > N1,t-,t+), (gluino(2) > N1,b,b)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (N1, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.90E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.37E-03 [pb]

 		 

		 .........................................................................

		 SMS ID: 62

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > N1,b,b), (gluino(2) > N1,b,b)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (N1, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:4.72E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:7.11E-03 [pb]

 		 


Topology: 11101010011011010000 

Total Global topology weight

In [23]:
for l in lines[876:900]:
    print(l)


Topology: 111010110101000110101101010000 

Total Global topology weight :

Sqrts: 1.30E+01 [TeV], Weight:1.24E+00 [pb]

Sqrts: 8.00E+00 [TeV], Weight:1.86E-01 [pb]



Total Number of SMS: 27

		 .........................................................................

		 SMS ID: 79

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > t-,b,C1+(3)), (gluino(2) > t-,b,C1+(4)), (C1+(3) > N1,q,q), (C1+(4) > N1,q,q)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N1, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:1.24E+00 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.86E-01 [pb]

 		 

		 .........................................................................

		 SMS ID: 80

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > t-,b,C1+(3)), (gluino(2) > t-,b,C1+(4)), (C1+(3) > N1,q,q), (C1+(4) > N1,q,c)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (C1+, 1.34E+02 [GeV]),

In [24]:
for l in lines[900:1099]:
    print(l)

		 SMS ID: 81

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > t-,b,C1+(3)), (gluino(2) > b,b,N2(4)), (C1+(3) > N1,q,q), (N2(4) > N1,q,q)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:9.82E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.48E-02 [pb]

 		 

		 .........................................................................

		 SMS ID: 82

		 SMS: (PV > gluino(1),gluino(2)), (gluino(1) > t-,b,C1+(3)), (gluino(2) > b,b,N2(4)), (C1+(3) > N1,q,q), (N2(4) > N1,c,c)

		 Masses: [(gluino, 5.77E+02 [GeV]), (gluino, 5.77E+02 [GeV]), (C1+, 1.34E+02 [GeV]), (N2, 1.35E+02 [GeV]), (N1, 6.81E+01 [GeV]), (N1, 6.81E+01 [GeV])]

		 Cross-Sections:

 		 Sqrts: 1.30E+01 [TeV], Weight:2.73E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.12E-03 [pb]

 		 

		 .........................................................................

		 SMS I

In [25]:
for l in lines[1099:1137]:
    print(l)


||                                                        ||

||                 Theory Predictions and                 ||

||                Experimental Constraints                ||

||                                                        ||




---------------Analysis Label = ATLAS-CONF-2013-037

-------------------Dataset Label = SRbC3

-------------------Txname Labels = ['T1bbtt']

Analysis sqrts: 8.00E+00 [TeV]

Theory prediction: 4.37E-05 [pb]

Theory conditions:  None

Observed experimental limit: 4.00E-01 [fb]

Expected experimental limit: 4.33E-01 [fb]

Observed r-value: 0.1093

Expected r-value: 0.101

Likelihood: 0.01901

L_max: 0.02067   -2log(L/L_max): 0.1681

L_SM: 0.02067   -2log(L/L_SM): 0.1681

Contributing SMS: [61]



---------------Analysis Label = ATLAS-SUSY-2013-02

-------------------Dataset Label = (UL)

-------------------Txname Labels = ['T1']

Analysis sqrts: 8.00E+00 [TeV]

Theory prediction: 1.53E+00 [pb]

Theory conditions:  None

Observed experimenta

In [26]:
for l in lines[1137:]:
    print(l)



Total cross-section for missing topologies (fb):  5.223E+03

Total cross-section for missing topologies with displaced decays (fb):  0.000E+00

Total cross-section for missing topologies with prompt decays (fb):  5.223E+03

Total cross-section for topologies outside the grid (fb):  4.719E+01



#Full information on unconstrained cross sections


missing topologies with the highest cross sections (up to 10):

Sqrts (TeV)   Weight (fb)                  SMS description

 13.0          1.204E+03    #                        PV > (jet,jet,MET), (nu,l,MET) 

Contributing SMS [31, 36, 10, 18, 8, 9, 16, 17, 29, 30, 34, 35, 32, 37, 11, 19]

 13.0          6.003E+02    #                       PV > (jet,jet,MET), (nu,ta,MET) 

Contributing SMS [41, 26, 24, 25, 39, 40, 42, 27]

 13.0          5.075E+02    #                         PV > (jet,jet,MET), (b,b,MET) 

Contributing SMS [52, 60]

 13.0          5.021E+02    #             PV > (b,t,jet,jet,MET), (b,b,jet,jet,MET) 

Contributing SMS [81, 8

In [27]:
masterCopy.outputFormat = "version2"
for printer in masterCopy.Printers.values():
    printer.outputFormat = "version2"

masterCopy.flush()    
with open(masterCopy.Printers['log'].filename,'r') as f:
    lines = f.readlines()
os.remove(masterCopy.Printers['log'].filename)        

In [28]:
print(len(lines))

1937


In [29]:
for l in lines[0:21]:
    if 'Database version' in l:
        continue        
    print(l)

Input status: 1

Decomposition output status: 1 #decomposition was successful

# Input File: ../testFiles/slha/lightEWinos.slha

# checkinput = True

# combineanas = ATLAS-CONF-2013-037,CMS-SUS-13-012

# combinesrs = False

# computestatistics = True

# docompress = True

# doinvisible = True

# experimentalfeatures = False

# maxcond = 0.2

# minmassgap = 5.

# model = share.models.mssm

# ncpus = 1

# promptwidth = 1e-11

# reportallsrs = False

# sigmacut = 10.0

# stablewidth = 1e-25

# testcoverage = True




In [30]:
for l in lines[21:205]:
    print(l)


||                                                        ||

||             Selected Experimental Results              ||

||                                                        ||



Experimental Result ID: ATLAS-SUSY-2016-08

Tx Labels: ['T5Disp']

Sqrts: 1.30E+01

	 -----------------------------

	 Elements tested by analysis:

	    [[[q,q]],[[q,q]]] 


Experimental Result ID: ATLAS-SUSY-2016-081

Tx Labels: ['T5Disp']

Sqrts: 1.30E+01

	 -----------------------------

	 Elements tested by analysis:

	    [[[q,q]],[[q,q]]] 


Experimental Result ID: ATLAS-SUSY-2018-31

Tx Labels: ['T6bbHH']

Sqrts: 1.30E+01

	 -----------------------------

	 Elements tested by analysis:

	    [[[b],[higgs]],[[b],[higgs]]] 


Experimental Result ID: CMS-PAS-EXO-16-036

Tx Labels: ['THSCPM1', 'THSCPM2', 'THSCPM3', 'THSCPM4', 'THSCPM5', 'THSCPM6', 'THSCPM7', 'THSCPM8']

Sqrts: 1.30E+01

	 -----------------------------

	 Elements tested by analysis:

	    [[],[]] 

	    [[[*anySM]],[]] 

	    [[[

In [31]:
for l in lines[205:300]:        
    print(l)


||                                                        ||

||                    Topologies Table                    ||

||                                                        ||



Topology: [][2] 

Total Global topology weight :

Sqrts: 1.30E+01 [TeV], Weight:3.50E-01 [pb]

Sqrts: 8.00E+00 [TeV], Weight:2.06E-01 [pb]



Total Number of Element: 4

		 .........................................................................

		 Element: 

		 Element ID: 1

		 Particles in element: [[],[[q,q]]]

		 Final states in element: [N1,N1/N1~]

		 The element masses are 

		 Branch 0: [6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000022], [[1000024, -1000024], [1000022, -1000022]]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:3.50E-01 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:2.06E-01 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 2

		 Part

In [32]:
il = 300
for l in lines[il:il+100]:        
    print(l)
il += 100

		 Element: 

		 Element ID: 6

		 Particles in element: [[[e-,nu]],[[nu,mu+]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.41E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.29E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 7

		 Particles in element: [[[e-,nu]],[[nu,ta+]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.41E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.29E-02 [pb]

 		 

		 .......

In [33]:
for l in lines[il:il+100]:        
    print(l)
il += 100

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.87E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.45E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 13

		 Particles in element: [[[mu-,nu]],[[nu,e+]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.41E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.29E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 14

		 Particles in element: [[[mu-,nu]],[[nu,mu+]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The e

In [34]:
for l in lines[il:il+100]:        
    print(l)
il += 100

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000023, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.30E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.17E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 20

		 Particles in element: [[[mu-,nu]],[[b,b]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000023, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.87E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.45E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 21

		 Particles in element: [[[ta-,nu]],[[nu,e+]]]

In [35]:
for l in lines[il:il+100]:        
    print(l)
il += 100

		 Element: 

		 Element ID: 26

		 Particles in element: [[[ta-,nu]],[[q,q]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000023, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:8.26E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.18E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 27

		 Particles in element: [[[ta-,nu]],[[c,c]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000023, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.30E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.16E-02 [pb]

 		 

		 .........

In [36]:
for l in lines[il:il+100]:        
    print(l)
il += 100

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:4.07E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:2.24E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 33

		 Particles in element: [[[nu,e+]],[[b,b]]]

		 Final states in element: [N1,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000023, 1000022], [1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:5.07E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:2.79E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 34

		 Particles in element: [[[nu,mu+]],[[q,q]]]

		 Final states in element: [N1,N1~]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PI

In [37]:
for l in lines[il:il+100]:        
    print(l)
il += 100

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:7.70E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.13E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 40

		 Particles in element: [[[nu,ta+]],[[q,c]]]

		 Final states in element: [N1,N1~]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:7.70E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.13E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 41

		 Particles in element: [[[nu,ta+]],[[q,q]]]



In [38]:
for l in lines[il:il+100]:        
    print(l)
il += 100

		 Element: 

		 Element ID: 46

		 Particles in element: [[[q,q]],[[e-,e+]]]

		 Final states in element: [N1/N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000023, 1000022], [[1000024, -1000024], [1000022, -1000022]]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.96E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.58E-02 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 47

		 Particles in element: [[[q,q]],[[mu-,mu+]]]

		 Final states in element: [N1/N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000023, 1000022], [[1000024, -1000024], [1000022, -1000022]]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.96E-02 [pb]

 		 Sqrts: 8.00E+

In [39]:
for l in lines[il:1152]:        
    print(l)

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.54E-01 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.36E-01 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 53

		 Particles in element: [[[q,c]],[[q,c]]]

		 Final states in element: [N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[-1000024, -1000022], [1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:2.47E-01 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.32E-01 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 54

		 Particles in element: [[[q,c]],[[e-,e+]]]

		 Final states in element: [N1/N1~,N1]

		 The element masses are 

		 Branch 0: [1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [1.35E+02 [GeV], 6.81E+01 [GeV]]



		 The element

In [40]:
for l in lines[1152:1420]:
    print(l)

 		 Sqrts: 8.00E+00 [TeV], Weight:7.11E-03 [pb]

 		 


Topology: [2][1,1] 

Total Global topology weight :

Sqrts: 1.30E+01 [TeV], Weight:3.19E-02 [pb]

Sqrts: 8.00E+00 [TeV], Weight:4.80E-03 [pb]



Total Number of Element: 1

		 .........................................................................

		 Element: 

		 Element ID: 63

		 Particles in element: [[[b,b]],[[g],[Z]]]

		 Final states in element: [N1,N1]

		 The element masses are 

		 Branch 0: [5.77E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [5.77E+02 [GeV], 2.66E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000021, 1000022], [1000021, 1000025, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:3.19E-02 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:4.80E-03 [pb]

 		 


Topology: [2][2,2] 

Total Global topology weight :

Sqrts: 1.30E+01 [TeV], Weight:4.91E-01 [pb]

Sqrts: 8.00E+00 [TeV], Weight:7.40E-02 [pb]



Total Number of Element: 7

		 ........................................

In [41]:
for l in lines[1420:1832]:
    print(l)

 		 Sqrts: 8.00E+00 [TeV], Weight:3.06E-03 [pb]

 		 


Topology: [2,2][2,2] 

Total Global topology weight :

Sqrts: 1.30E+01 [TeV], Weight:1.24E+00 [pb]

Sqrts: 8.00E+00 [TeV], Weight:1.86E-01 [pb]



Total Number of Element: 27

		 .........................................................................

		 Element: 

		 Element ID: 79

		 Particles in element: [[[t-,b],[q,q]],[[t-,b],[q,q]]]

		 Final states in element: [N1,N1]

		 The element masses are 

		 Branch 0: [5.77E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]

		 Branch 1: [5.77E+02 [GeV], 1.34E+02 [GeV], 6.81E+01 [GeV]]



		 The element PIDs are 

		 PIDs: [[1000021, 1000024, 1000022], [1000021, 1000024, 1000022]]

		 The element weights are: 

 		 Sqrts: 1.30E+01 [TeV], Weight:1.24E+00 [pb]

 		 Sqrts: 8.00E+00 [TeV], Weight:1.86E-01 [pb]

 		 

		 .........................................................................

		 Element: 

		 Element ID: 80

		 Particles in element: [[[t-,b],[q,q]],[[t-,b],[q,c]]]

		 Final

In [42]:
for l in lines[1832:1875]:
    print(l)

 		 Sqrts: 8.00E+00 [TeV], Weight:2.88E-03 [pb]

 		 


||                                                        ||

||                 Theory Predictions and                 ||

||                Experimental Constraints                ||

||                                                        ||




---------------Analysis Label = ATLAS-CONF-2013-037

-------------------Dataset Label = SRbC3

-------------------Txname Labels = ['T1bbtt']

Analysis sqrts: 8.00E+00 [TeV]

Theory prediction: 4.37E-05 [pb]

Theory conditions:  None

Observed experimental limit: 4.00E-01 [fb]

Expected experimental limit: 4.33E-01 [fb]

Observed r-value: 0.1093

Expected r-value: 0.101

Likelihood: 0.01901

L_max: 0.02067   -2log(L/L_max): 0.1681

L_SM: 0.02067   -2log(L/L_SM): 0.1681

Contributing Elements: [61]

PIDs:[[1000021, 1000022], [1000021, 1000022]]



---------------Analysis Label = ATLAS-SUSY-2013-02

-------------------Dataset Label = (UL)

-------------------Txname Labels = ['T1']

Analy

In [43]:
for l in lines[1875:]:
    print(l)





Total cross-section for missing topologies (fb):  5.223E+03

Total cross-section for missing topologies with displaced decays (fb):  0.000E+00

Total cross-section for missing topologies with prompt decays (fb):  5.223E+03

Total cross-section for topologies outside the grid (fb):  4.719E+01



#Full information on unconstrained cross sections


missing topologies with the highest cross sections (up to 10):

Sqrts (TeV)   Weight (fb)                  Element description

 13.0          1.204E+03    #                      [[[jet,jet]],[[l,nu]]] (MET,MET) 

Contributing Element [31, 36, 10, 18, 8, 9, 16, 17, 29, 30, 34, 35, 32, 37, 11, 19]

 13.0          6.003E+02    #                     [[[jet,jet]],[[ta,nu]]] (MET,MET) 

Contributing Element [41, 26, 24, 25, 39, 40, 42, 27]

 13.0          5.075E+02    #                       [[[jet,jet]],[[b,b]]] (MET,MET) 

Contributing Element [52, 60]

 13.0          5.021E+02    #           [[[jet,jet,t,b]],[[jet,jet,b,b]]] (MET,MET) 

Contr